In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from huggingface_hub import login, whoami

login()

In [3]:
# DATASET CONFIG
DATASET='tyqiangz/multilingual-sentiments'
MODEL='lxyuan/distilbert-base-multilingual-cased-sentiments-student'
MIN_TOKENS=50
MAX_TOKENS=512
SAMPLES=10_000
MAX_VUS = 2000
TEXT_COLUMN='text'
DATASET_PATH='data/dataset.json'
# K6_BIN = "/usr/bin/k6"
K6_BIN = "~/.local/bin/k6-sse"

HUB_DATASET_PATH = f'{whoami()["name"]}/classifier-ie-optimization'

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer
from src.process_dataset import tokenize_and_filter, sample_dataset, save_dataset

dataset = load_dataset(DATASET, 'all', split='train')

# Map numbers to text labels
label_mapping = {0: 'positive', 1: 'neutral', 2: 'negative'}
dataset = dataset.map(lambda example: {'label_text': label_mapping[example['label']]})


tokenizer = AutoTokenizer.from_pretrained(MODEL)

dataset = tokenize_and_filter(dataset, tokenizer, text_column=TEXT_COLUMN, num_proc=8)
dataset = sample_dataset(dataset, n_samples=SAMPLES, min_tokens=MIN_TOKENS, max_tokens=MAX_TOKENS, seed=42)
save_dataset(dataset.select_columns([TEXT_COLUMN, 'label_text']), DATASET_PATH)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
2025-01-25 21:08:07.467 | SUCCESS  | src.process_dataset:sample_dataset:31 - Sampled dataset down to 10000 samples
2025-01-25 21:08:07.525 | SUCCESS  | src.process_dataset:save_dataset:44 - Saved dataset to data/dataset.json


In [5]:
from dataclasses import dataclass, field, asdict
from typing import Dict

@dataclass
class InstanceConfig:
    repository: str
    accelerator: str
    instance_size: str
    instance_type: str
    custom_image: Dict[str, str]  # Dict of str -> str
    vendor: str
    region: str

@dataclass
class InfinityConfig:
    INFINITY_PORT: str = "80"
    INFINITY_ENGINE: str = "torch"
    INFINITY_BATCH_SIZE: str = "16"
    INFINITY_DTYPE: str = "auto"
    INFINITY_EMBEDDING_DTYPE: str = "float32"
    INFINITY_POOLING_METHOD: str = "auto"
    INFINITY_COMPILE: str = 'true'
    INFINITY_BETTERTRANSFORMER: str = 'true'
    INFINITY_MODEL_ID: str = "/repository"

    def to_dict(self) -> Dict[str, str]:
        """Converts the dataclass to a dictionary representation."""
        return asdict(self)

@dataclass
class ImageConfig:
    health_route: str = "/health"
    url: str = "michaelf34/infinity:0.0.75"
    env: InfinityConfig = field(default_factory=InfinityConfig)


In [6]:
def set_env(batch_size: int, image_config: ImageConfig) -> Dict[str, str]:
    config = asdict(image_config)
    config['env']["INFINITY_BATCH_SIZE"] = str(batch_size)
    return config

# Experiments

In [7]:
from src.deployment import deploy_endpoint
?deploy_endpoint

Signature: deploy_endpoint(instance_config, endpoint_name, wait=False)
Docstring: Creates and deploys an inference endpoint using the given instance configuration.
File:      ~/projects/encoder-analysis/src/deployment.py
Type:      function

## GPUs

In [8]:
instance_config_experiment_dicts = [
    # {
    #     'accelerator': 'gpu',
    #     'instance_size': 'x1',
    #     'instance_type': 'nvidia-l4',
    #     'vendor': 'gcp',
    #     'region': 'us-east4',
    # },
    {
        'accelerator': 'gpu',
        'instance_size': 'x1',
        'instance_type': 'nvidia-t4',
        'vendor': 'aws',
        'region': 'us-east-1',
    },
    {
        'accelerator': 'gpu',
        'instance_size': 'x1',
        'instance_type': 'nvidia-l4',
        'vendor': 'aws',
        'region': 'us-east-1',
    },
]
instance_config_experiments = [InstanceConfig(repository=MODEL, **instance_config_experiment_dict, custom_image=None) for instance_config_experiment_dict in instance_config_experiment_dicts]

In [9]:
import copy
from pathlib import Path
from time import sleep

from loguru import logger

from src.k6 import call_k6, optimal_vus

template_file = "classification-analysis.js.j2"
output_file = Path("./generated").resolve()/"classification-analysis.js"

start_vus = 32
batch_sizes = [16, 32, 64, 128, 256, 512, 1024]

for og_instance_config_experiment in instance_config_experiments:
    endpoints = []
    for batch_size in batch_sizes:
        # Configure Infinity Settings, pass them up to the image
        infinty_config = InfinityConfig(INFINITY_BATCH_SIZE=str(batch_size))
        image_config = ImageConfig(env=infinty_config)

        # Add the image to the HW instance
        instance_config_experiment = copy.deepcopy(og_instance_config_experiment)
        instance_config_experiment.custom_image = image_config

        # Deploy the endpoint
        logger.info(f'Creating endpoint with Batch Size:\t{batch_size}')
        vendor = instance_config_experiment.vendor
        endpoint = deploy_endpoint(instance_config_experiment, endpoint_name=f'ea-{vendor}-{batch_size}', wait=False)
        endpoints.append(endpoint)

    endpoints[0].wait()
    
    for endpoint in endpoints:
        endpoint.fetch()
        batch_size = endpoint.__dict__["raw"]["model"]["env"]["INFINITY_BATCH_SIZE"]
        vendor = endpoint.__dict__["raw"]["provider"]["vendor"]
        instance_type = endpoint.__dict__["raw"]["compute"]["instanceType"]
        logger.success(f'Endpoint is ready!')
        logger.info(f'\tVendor: {vendor}')
        logger.info(f'\tInstance: {instance_type}')
        logger.info(f'\tBatch Size: {batch_size}')

        args_dict = dict(
            endpoint=endpoint,
            total_requests=10_000,
            template_file=template_file,
            output_file=output_file,
            dataset_path=DATASET_PATH,
            k6_bin=K6_BIN
        )

        optimal_vus(max_vus=MAX_VUS, args_dict=args_dict, start_vus=start_vus)
        endpoint.delete()
        sleep(5)

2025-01-25 21:08:07.614 | INFO     | __main__:<module>:27 - Creating endpoint with Batch Size:	16
2025-01-25 21:08:08.995 | INFO     | src.deployment:deploy_endpoint:21 - Creating inference endpoint...
2025-01-25 21:08:09.321 | INFO     | src.deployment:deploy_endpoint:40 - Waiting for endpoint to be ready...
2025-01-25 21:08:09.324 | INFO     | __main__:<module>:27 - Creating endpoint with Batch Size:	32
2025-01-25 21:08:09.830 | SUCCESS  | src.deployment:deploy_endpoint:15 - Re-using Endpoint: hw=nvidia-t4	bs=32	
2025-01-25 21:08:09.832 | INFO     | __main__:<module>:27 - Creating endpoint with Batch Size:	64
2025-01-25 21:08:10.433 | SUCCESS  | src.deployment:deploy_endpoint:15 - Re-using Endpoint: hw=nvidia-t4	bs=64	
2025-01-25 21:08:10.435 | INFO     | __main__:<module>:27 - Creating endpoint with Batch Size:	128
2025-01-25 21:08:11.007 | SUCCESS  | src.deployment:deploy_endpoint:15 - Re-using Endpoint: hw=nvidia-t4	bs=128	
2025-01-25 21:08:11.009 | INFO     | __main__:<module>:27

In [11]:
from src.process_dataset import load_json_files

# Define cost per hour considering both hardware type and vendor
cost_per_hour = {
    ('gcp', 'nvidia-t4'): 0.5,
    ('gcp', 'nvidia-l4'): 0.7,
    ('aws', 'nvidia-t4'): 0.5,
    ('aws', 'nvidia-l4'): 0.8,
}

# Load dataset
df = load_json_files('results')

# Compute cost
df['seconds_per_billion'] = 1e9 / df['throughput_req_per_sec']
df['cost_per_sec'] = df.apply(lambda row: cost_per_hour.get((row['vendor'], row['hw_type']), 0) / 3600, axis=1)
df['1B_cost'] = df['seconds_per_billion'] * df['cost_per_sec']

df.sort_values(by='1B_cost').head()

,total_requests,test_duration_sec,successful_requests,avg_latency_ms,p95_latency_ms,accuracy_percentage,throughput_req_per_sec,hw_type,batch_size,engine,vendor,vus,seconds_per_billion,cost_per_sec,1B_cost
22,10000,13.482449,10000,763.1916,1064.05,58.04,741.705012,nvidia-l4,256,torch,aws,575,1348244.9,0.000222,299.609978
40,10000,13.511792,10000,682.2363,1301.00,58.03,740.094282,nvidia-l4,256,torch,aws,512,1351179.2,0.000222,300.262044
39,10000,13.529694,10000,1019.6080,1812.10,58.04,739.115016,nvidia-l4,256,torch,aws,768,1352969.4,0.000222,300.659867
23,10000,13.548960,10000,786.3956,1112.05,58.05,738.064029,nvidia-l4,256,torch,aws,591,1354896.0,0.000222,301.088000
86,10000,13.552313,10000,897.3117,1249.00,58.04,737.881423,nvidia-l4,64,torch,aws,675,1355231.3,0.000222,301.162511


In [12]:
from datasets import Dataset
# Push to the hub
dataset = Dataset.from_pandas(df)
dataset.push_to_hub(HUB_DATASET_PATH)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/862 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/derek-thomas/classifier-ie-optimization/commit/bed51f10e92d9bdd628880c0507c2429b0b567b1', commit_message='Upload dataset', commit_description='', oid='bed51f10e92d9bdd628880c0507c2429b0b567b1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/derek-thomas/classifier-ie-optimization', endpoint='https://huggingface.co', repo_type='dataset', repo_id='derek-thomas/classifier-ie-optimization'), pr_revision=None, pr_num=None)

## CPU Experiments

In [ ]:
instance_config_experiment_dicts = [
    {
        'accelerator': 'cpu',
        'instance_size': 'x1',
        'instance_type': 'intel-spr'
    }
]

instance_config_experiments = [InstanceConfig(repository=MODEL, **ic_exp_dict, custom_image=None) for ic_exp_dict in instance_config_experiment_dicts]

In [ ]:
from itertools import product

batch_sizes = [1, 2, 4, 8, 16, 32, 64]
infinity_engine = ['optimum', 'torch']
start_vus = 1

cross_product = list(product(batch_sizes, infinity_engine))

for og_instance_config_experiment in instance_config_experiments:
    for batch_size, infinity_engine in cross_product:
        # Configure Infinity Settings, pass them up to the image
        infinty_config = InfinityConfig(INFINITY_BATCH_SIZE=str(batch_size))
        image_config = ImageConfig(env=infinty_config)

        # Add the image to the HW instance
        instance_config_experiment = copy.deepcopy(og_instance_config_experiment)
        instance_config_experiment.custom_image = image_config

        # Deploy the endpoint
        logger.info(f'Deploying with Batch Size:\t{batch_size}')
        logger.info(f'Deploying with Infinity Engine:\t{infinity_engine}')
        endpoint = deploy_endpoint(instance_config_experiment)

        args_dict = dict(
            endpoint=endpoint,
            total_requests=10_000,
            template_file=template_file,
            output_file=output_file,
            dataset_path=DATASET_PATH,
            k6_bin=K6_BIN
        )
        
        optimal_vus(max_vus=CPU_MAX_VUS, args_dict=args_dict, start_vus=start_vus)
        endpoint.delete()
        sleep(5)

In [18]:
import subprocess
from huggingface_hub import get_token
import os

K6_BIN = os.path.expanduser(K6_BIN)  # Ensure correct path
process = subprocess.run(
        [K6_BIN, "run", str(output_file)],
        env={'HF_TOKEN': get_token(), **os.environ},
        capture_output=True,
        text=True
)